In [4]:
import requests
import pandas as pd
from lxml import etree

In [9]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
res = requests.get(url)
tree = etree.HTML(res.text)
trs = tree.xpath('//table[@class="wikitable sortable"]/tbody/tr')
arr = []
for tr in trs:
    info = tr.xpath('.//td')
    if len(info) == 0:
        continue
    else:
        arr.append(str(tr.xpath('string()')).strip('\n').split('\n'))
    # print(etree.tostring(info))
df = pd.DataFrame(arr,index=list(range(0,len(arr))),columns=list(['Postcode','Borough','Neighbourhood']))
df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
283,M8Z,Etobicoke,Mimico NW
284,M8Z,Etobicoke,The Queensway West
285,M8Z,Etobicoke,Royal York South West
286,M8Z,Etobicoke,South of Bloor
